### Trying out RAG with ollama and chromadb
ollama is installed in the python environment venvcrc5 from where this notebook is started.

ollama is recommended over hugging face for local experimentation

it uses a docker-like syntax

In [1]:
!ollama --version
!ollama list

ollama version is 0.5.7
NAME                       ID              SIZE      MODIFIED     
deepseek-r1:latest         0a8c26691023    4.7 GB    45 hours ago    
nomic-embed-text:latest    0a109f422b47    274 MB    45 hours ago    
llama3.1:latest            46e0c10c039e    4.9 GB    4 days ago      


#### The pdfreader translates any pdf document to text readable by the model

In [16]:
from pypdf import PdfReader
# my textbook, 5th ed
reader = PdfReader("/home/mort/LaTeX/new projects/CRC5/main.pdf")
total_pages = len(reader.pages)
all_text = ""
for page_num in range(total_pages):
    page = reader.pages[page_num]
    all_text += page.extract_text()
f = open("/home/mort/temp/main.txt", "w")
f.write(all_text)
f.close()

#### Code for preprocessing the RAG supplementary text

In [3]:
import os
import re
import ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter

def readtextfiles(path):
    text_contents = {}
    directory = os.path.join(path)
    
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory, filename)
        
            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()
        
            text_contents[filename] = content
        
        return text_contents

def chunksplitter(text, chunk_size=256, chunk_overlap=50):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,  # Desired chunk size in characters or tokens
        chunk_overlap=chunk_overlap,  # Overlap between chunks
        separators=["\n\n", "\n", " "]  # Split by paragraphs, then sentences, then words
    )
    return splitter.split_text(text)

# use the nomic-embed-text model to calculate vector embeddings for all text chunks
def getembedding(chunks):
    embeds = ollama.embed(model="nomic-embed-text", input=chunks)
    return embeds.get('embeddings', [])

#### Add the supplementary text to a new database collection

In [4]:
import chromadb
chromaclient = chromadb.PersistentClient(path="/home/mort/crc5imagery/crc5rag")
# erase any existing collection and create a new empty one
chromaclient.delete_collection(name="crc5rag")
collection = chromaclient.create_collection(name="crc5rag", metadata={"hnsw:space": "cosine"}  )

# the RAG supplementary data
textdocspath = "/home/mort/temp"
text_data = readtextfiles(textdocspath)

# read, break into chunks, embed and add to the chroma vector database 
for filename, text in text_data.items():
    # chunk size default 256, overlap default 50
    chunks = chunksplitter(text,512,100)
    embeds = getembedding(chunks)
    chunknumber = list(range(len(chunks)))
    ids = [filename + str(index) for index in chunknumber]
    metadatas = [{"source": filename} for index in chunknumber]
    collection.add(ids=ids, documents=chunks, embeddings=embeds, metadatas=metadatas)


#### Execute a query with llama3.1 or deepseek-r1 and the supplementary text (RAG)

In [3]:
#%%capture
!ollama pull nomic-embed-text
#!ollama pull llama3.1
!ollama pull deepseek-r1

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 
pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 96c415656d37... 100% ▕████████████████▏ 4.7 GB                         
pulling 369ca498f347... 100% ▕████████████████▏  387 B                         
pulling 6e4c38e1172f... 100% ▕████████████████▏ 1.1 KB                         
pulling f4d24e9138dd... 100% ▕████████████████▏  148 B                         
pulling 40fb844194b2... 100% ▕████████████████▏  487 B                         
verifying sha256 dige

In [5]:
import gradio as gr
import ollama
import chromadb

chromaclient = chromadb.PersistentClient(path="/home/mort/crc5imagery/crc5rag")
collection = chromaclient.get_collection(name="crc5rag")

def ragask(query):
    # embed the current query
    queryembed = ollama.embed(model="nomic-embed-text", input=query)['embeddings']
    
    # use the embedded current query to retrieve the most relevant document chunks (as text NOT as embeddings)
    relateddocs = '\n\n'.join(collection.query(query_embeddings=queryembed, n_results=4)['documents'][0])

    # generate an answer
    prompt = f"Answer the question: {query}, referring to the following text as a resource: {relateddocs}"
    ragoutput = ollama.generate(model="deepseek-r1", prompt=prompt, stream=False)

    return ragoutput['response']

# use the gradio interface
gr.Interface(fn=ragask, inputs="text", outputs="text").launch()   

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
